In [39]:
import cx_Oracle
import pandas as pd
import requests
from xml.etree.ElementTree import fromstring
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager,rc
import pyproj

In [40]:
conn = cx_Oracle.connect('open_source/1111@localhost:1521/xe')
cur = conn.cursor()
cur.execute("""
select column_name from user_tab_columns where table_name = 'BUS_753'
""")

<cx_Oracle.Cursor on <cx_Oracle.Connection to open_source@localhost:1521/xe>>

In [41]:
column_list = [record[0] for record in cur]
column_list

['정류소ID', '정류소명', '첫번째도착정보메시지', '두번째도착정보메시지', '제공시각']

In [42]:
cur.execute('select * from BUS_753')
bus753_df= pd.DataFrame(cur, columns=column_list)
bus753_df.head()

,정류소ID,정류소명,첫번째도착정보메시지,두번째도착정보메시지,제공시각
0,111000299,구산동사거리,출발대기,출발대기,2022-12-28 17:20:45
1,111000181,한솔아파트입구선정중학교후문,출발대기,출발대기,2022-12-28 17:20:45
2,111000182,갈현동미미아파트,곧 도착,출발대기,2022-12-28 17:20:45
3,111000183,선일여고입구,2분26초후[1번째 전],출발대기,2022-12-28 17:20:45
4,111000184,갈현동성당앞,3분41초후[2번째 전],출발대기,2022-12-28 17:20:45


In [43]:
bus753_df['정류소ID']=bus753_df['정류소ID'].astype('int')

In [44]:
bus753_df.dtypes

정류소ID                  int32
정류소명                  object
첫번째도착정보메시지            object
두번째도착정보메시지            object
제공시각          datetime64[ns]
dtype: object

In [45]:
bus753_df.iloc[0][0]

111000299

In [46]:
font_location = "C:\Windows\Fonts\malgun.ttf"
font_name = font_manager.FontProperties(fname=font_location).get_name()
rc('font',family=font_name)
plt.rcParams['axes.unicode_minus'] = False
bus_station_df = pd.read_csv('서울시버스정류소좌표데이터.csv', encoding='cp949')

In [47]:
bus_station_df.head()

,정류소ID,ARS-ID,정류소명,X좌표,Y좌표
0,100000001,1001,종로2가사거리,126.987750,37.569765
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671
3,100000004,1004,종로2가.삼일교,126.987613,37.568579
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243


In [48]:
bus_station_df.dtypes

정류소ID       int64
ARS-ID      int64
정류소명       object
X좌표       float64
Y좌표       float64
dtype: object

In [49]:
bus_station_df.iloc[0][0]

100000001

In [50]:
bus_station_df=bus_station_df[['정류소ID','X좌표','Y좌표']]

In [51]:
df2 = pd.merge(left = bus753_df , right = bus_station_df, how = "inner", on = "정류소ID")

In [52]:
df2.head()

,정류소ID,정류소명,첫번째도착정보메시지,두번째도착정보메시지,제공시각,X좌표,Y좌표
0,111000299,구산동사거리,출발대기,출발대기,2022-12-28 17:20:45,126.912234,37.614271
1,111000299,구산동사거리,출발대기,출발대기,2022-12-28 16:43:03,126.912234,37.614271
2,111000181,한솔아파트입구선정중학교후문,출발대기,출발대기,2022-12-28 17:20:45,126.913707,37.615796
3,111000181,한솔아파트입구선정중학교후문,출발대기,출발대기,2022-12-28 16:43:03,126.913707,37.615796
4,111000182,갈현동미미아파트,곧 도착,출발대기,2022-12-28 17:20:45,126.915152,37.618203


In [53]:
df2.shape

(208, 7)

In [77]:
def plt_update():
    cur.execute('select * from bus_753 where 제공시각 = (select max(제공시각) from bus_753)')

    columns = ["정류소ID","정류소명","첫번째도착정보메시지","두번째도착정보메시지","제공시각"]
    df_bus_now = pd.DataFrame(cur,columns=columns)
    df_bus_now.head()

In [78]:
plt_update()

In [ ]:
def plt_update(ax):
    cur.execute('select * from weather where 제공시각 = max(제공시각)')

    columns = ['DATE_TIME','NX', 'NY', '시간1_강수량', '강수형태', '기온', 
                         '습도', '풍향', '풍속', '동서바람성분', '남북바람성분']
    df = pd.DataFrame(cur,columns=columns)
    ax.plot(df['DATE_TIME'],df['풍속'], color='b')
    
    hdisplay.update(fig)

In [ ]:
fig,ax = plt.subplots(1,1)
plt.xticks(rotation=80)
hdisplay = display.display("", display_id=True)

ax.set_xlabel('날짜')
ax.set_ylabel('측정값')

# plt_update(ax)

for f in range(300):
    plt_update(ax)
    time.sleep(1)

plt.close()